# Michlmayr
Only virus data! 

## Data processing	
Sequencer-generated base call (BCL) files were converted to FASTQ files and the multiplexed samples were separated using bcl2fastq
Demultiplexed fastq were assessed for sequencing quality using FastQC (version 0.11.4, http://www.bioinformatics.babraham.ac.uk/projects/fastqc/)
The fastq files were quality filtered by using FASTAX Toolkit (http://hannonlab.cshl.edu/fastx_toolkit/) with the invocation fastq_quality_filter -q 30 -p 50 -v -Q 33
Sequencing reads that met all quality control requirements were aligned to the latest human reference genome (GRCh38) using HISAT2 (version 2.0.4)

Gene expression in units of overlapping read counts per million were obtained by using the htseq-count script from HTseq on SAM files of preprocessed RNA-seq alignments.
Counts per million were normalized using the voom methodology and expression profiles were modeled using mixed-effect models in limma (version 3.30.6)
Genome_build: GRCh38


Supplementary_files_format_and_content: csv file with the following output: lgFCH_AcuteVsConv: log2 of the fold change in expression between acute and convalescent timepoints, FCH_AcuteVsConv: fold change in expression between acute and convalescent timepoints, pvals_AcuteVsConv, fdrs_AcuteVsConv, StatusFCH2FDR0.05_AcuteVsConv, a cutoff for |fold change| >2 and FDR <0.05. It is -1 if the gene is fold change <-2 and FDR <0.05, and it is +1 if the gene is fold change >2 and FDR <0.05. Otherwise it is 0.

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt

root_path = '/var/datasets/single_papers/2018_michlmayr'


In [2]:
metadata = read_prefixes_from_metadata_txt(f'{root_path}/GSE99992_series_matrix.txt')
print(metadata.shape)
metadata.head()

(172, 46)


/home/eliasschreiner/PROJEKT/utils/geo_utils.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,...,contact_zip/postal_code,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,relation.2,supplementary_file_1
0,"Patient 1, acute sample, technical replicate 1",GSM2667266,Public on Jun 13 2018,Jun 13 2017,May 15 2019,SRA,1,Whole blood,Homo sapiens,timepoint: 1-2d post symptom onset,...,10029,USA,0,Illumina HiSeq 4000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,NONE
1,"Patient 1, acute sample, technical replicate 2",GSM2667267,Public on Jun 13 2018,Jun 13 2017,May 15 2019,SRA,1,Whole blood,Homo sapiens,timepoint: 1-2d post symptom onset,...,10029,USA,0,Illumina HiSeq 4000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,NONE
2,"Patient 1, conv sample, technical replicate 1",GSM2667268,Public on Jun 13 2018,Jun 13 2017,May 15 2019,SRA,1,Whole blood,Homo sapiens,timepoint: 15-17d post symptom onset,...,10029,USA,0,Illumina HiSeq 4000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,NONE
3,"Patient 1, conv sample, technical replicate 2",GSM2667269,Public on Jun 13 2018,Jun 13 2017,May 15 2019,SRA,1,Whole blood,Homo sapiens,timepoint: 15-17d post symptom onset,...,10029,USA,0,Illumina HiSeq 4000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,NONE
4,"Patient 2, acute sample, technical replicate 1",GSM2667270,Public on Jun 13 2018,Jun 13 2017,May 15 2019,SRA,1,Whole blood,Homo sapiens,timepoint: 1-2d post symptom onset,...,10029,USA,0,Illumina HiSeq 4000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,NONE


In [3]:
raw_counts = pd.read_csv(f'{root_path}/GSE99992_BigTab.Timepoint.AllGenes.2017-04-25_Transcripts.csv')
print(raw_counts.shape)
raw_counts.head()

(53971, 10)


,Unnamed: 0,target_id,ens_gene,ext_gene,gene_biotype,lgFCH_AcuteVsConv,FCH_AcuteVsConv,pvals_AcuteVsConv,fdrs_AcuteVsConv,StatusFCH2FDR0.05_AcuteVsConv
0,ENST00000000412,ENST00000000412,ENSG00000003056,M6PR,protein_coding,-0.11,-1.08,4.990000e-01,5.750000e-01,0
1,ENST00000000442,ENST00000000442,ENSG00000173153,ESRRA,protein_coding,0.12,1.09,5.250000e-01,5.990000e-01,0
2,ENST00000001008,ENST00000001008,ENSG00000004478,FKBP4,protein_coding,0.39,1.31,2.510000e-04,6.140000e-04,0
3,ENST00000002125,ENST00000002125,ENSG00000003509,NDUFAF7,protein_coding,-0.04,-1.03,8.960000e-01,9.190000e-01,0
4,ENST00000002165,ENST00000002165,ENSG00000001036,FUCA2,protein_coding,0.42,1.33,1.890000e-25,3.290000e-24,0
